# Least Square

$$
\min_\beta ||A\beta - y||^2
$$

Where $A$ is the design matrix. Is a $n \times (p+1)$ matrix where each column is a feature and each row is an observation.

$$
\left[
\begin{matrix}
1 & x_1^1 & x_2^1 & \cdots & x_p^1 \\
1 & x_1^2 & x_2^2 & \cdots & x_p^2\\
\vdots & \vdots & \vdots && \vdots\\
1 & x_1^n & x_2^n & \cdots & x_p^n\\
\end{matrix}
\right]
$$

In [ ]:
# ordinary least squares is used to find linear coefficients with help of iris dataset.

# "Sepal.Length" and "Petal.Width" columns are used to predict "Petal.Length"
A = iris[ , c("Sepal.Length", "Petal.Width") ]

# 1s is used as column for the intercept coefficient
A[ , 'intercept'] = 1

In [ ]:
Y = iris[ , "Petal.Length" ]

In [ ]:
A = as.matrix(A)
Y = as.matrix(Y)

For statisticians, solving the least squares is usaully via

$$
\hat{\beta} = A^\dagger y
$$

where $A^\dagger$ is the pseudoinverse.

The pseudoinverse is $A^\dagger = (A^\top A) ^{-1} A^\top$ when $A$ is full rank, otherwsie it is also defined.

In [ ]:
ols1 = ( solve( t(A) %*% A ) %*% t(A) ) %*% Y
ols1

# comparing it with lm function
lm(Petal.Length ~ Sepal.Length + Petal.Width, data=iris)

## Gradient Descent

$$
\nabla(\beta) = 2 A^\top(A \beta - y).
$$

In [ ]:
lr = 0.0001  # learning rate
b = runif(3) # random starting points for beta
bh = t(b) # bh stores value of betas for each iteration
for (i in 1:100000) {
    gradient = 2 * t(A) %*% ( (A %*% b) - Y ) # calculate gradient
    b = b - lr * gradient # update beta
    bh = rbind(bh, t(b)) # store beta to bh
}
b

### Calculating Error for each combination of $\beta$ through gradient descent

Calculating $\sum (\hat{y}-y)^2 $ for each $\beta$ combination of gradient descent

In [ ]:
error = colSums((A %*% t(bh) - as.vector(Y))^2)

In [ ]:
# plotting the coefficients (in the 3 axes) - with error as color

gradientData = data.frame(bh, error=error)

library('plotly')
fig <- plot_ly(gradientData, x = ~Sepal.Length, y = ~Petal.Width, z = ~intercept, type = 'scatter3d', mode = 'lines',
        opacity = 1, line = list(width = 6, color = ~error, reverscale = FALSE))
fig

## Now the discussion goes into MNIST
MNIST contain 28 X 28 pixels hand-written digits like the below.<br>
$28\times 28 = 784$ features 

<div>
<img src="img/MNIST.png" width="200" align="center"/>
</div>

In [ ]:
library('keras')
library('MASS') # package to calculate pseudo inverse

In [ ]:
mnist <- dataset_mnist() # loading data -- function present in keras package

mnist dataset contains a training set and a test set.<br>
"$x$" and "$y$" are explanatory and response variables in each set.

In [ ]:
str(mnist)

In [ ]:
mnist$train$x[1,,]

#### Plotting the 1st digit of MNIST

In [ ]:
library("plot.matrix")
plot(mnist$train$x[1,,])

In [ ]:
# flattening 28 X 28 pixels

A <- array_reshape( mnist$train$x, c(dim(mnist$train$x)[1], 784) )
dim(A)
head(A)

In [ ]:
# adding 1s as a feature for the intercept term

A = cbind(matrix(rep(1, nrow(A))), A)
dim(A)
head(A)

In [ ]:
 # The response variable is the digit number varying from 0 to 9.
 
table(mnist$train$y)

In [ ]:
# to_categorical function is used to create binary matrix from categorical data.
# However it requires the categorical data to be numbered starting from 0. else it will create '0' columns for number from 0 until the minimum value of categorical column.

to_categorical(0:2)
to_categorical(1:3)

In [ ]:
# converting the response column to binary class matrix

Y = to_categorical(mnist$train$y) 
head(Y)

In [ ]:
# optional - replacing 0 with -1

Y[Y==0] = -1
dim(Y)
head(Y)

$$
\hat{\beta} = A^\dagger y
$$

where $A^\dagger$ is the pseudoinverse.

In [ ]:
pseudoInv = ginv(A)

In [ ]:
b = pseudoInv %*% Y
dim(b)

In [ ]:
plot(b[,9], ylim=c(-.2,.2))

$$
\hat{y} = A \hat{\beta}
$$


In [ ]:
pred = A %*% b
head(pred)

In [ ]:
# "pred" is a number proportional to the possibility of an instance belonging to a digit. 
# we classify the instance into that class which is maximum 

table(mnist$train$y, apply(pred, 1, which.max)-1)   #  confusion matrix
mean(mnist$train$y == apply(pred, 1, which.max)-1)  #  train accuracy

In [ ]:
# redoing the same steps to calculate test error

A <- array_reshape( mnist$test$x, c(dim(mnist$test$x)[1], 784) )
A = cbind(matrix(rep(1, nrow(A))), A)
pred = A %*% b

table(mnist$test$y, apply(pred, 1, which.max)-1)
mean(mnist$test$y == apply(pred, 1, which.max)-1)

#### Create a function which takes as argument an image, and returns the digit predicted using the trained $\beta$

In [ ]:
myClassifier = function(img) {
    #     convert img to vector
    #     append intercept
    #     multiply by trained weights
    #     choose which max
    #     return predicted digit
}

In [ ]:
myClassifier = function(img) {

    # convert img to vector
    A <- array_reshape( img, c(1, 784) )
    
    # append intercept
    A = cbind(matrix(rep(1, nrow(A))), A)

    # multiply by trained weights
    pred = A %*% b
    
    # choose which max
    predDigit = apply(pred, 1, which.max)-1
    
    return(predDigit)

}

In [ ]:
# predicting a digit using the classifier udf

myClassifier(mnist$train$x[1,,])
mnist$train$y[1]

### MNIST Prediction using Gradient Descent 

In [ ]:
A <- array_reshape( mnist$train$x, c(dim(mnist$train$x)[1], 784) )
A = cbind(matrix(rep(1, nrow(A))), A)

Y = to_categorical(mnist$train$y)
Y[Y==0] = -1 # optional


lr = 0.00001 
b = matrix(runif(ncol(A)*ncol(Y)), ncol=ncol(Y)) ;
for (i in 1:10) {
    gradient = 2 * t(A) %*% ( (A %*% b) - Y )
    b = b - lr * gradient
}
